In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib as plt

In [2]:
data=pd.read_csv("Score.csv")
#data

In [3]:
col=np.array(data.columns)
col=np.delete(col,range(3))
#col

In [4]:
'''
maxScore1=[100 for i in range(36)]
maxScore2=[150,150,150,100,150,150,150,100,60,150,150,150,100,100,150,150,150,90,60]
maxScore=maxScore1+maxScore2
'''

'\nmaxScore1=[100 for i in range(36)]\nmaxScore2=[150,150,150,100,150,150,150,100,60,150,150,150,100,100,150,150,150,90,60]\nmaxScore=maxScore1+maxScore2\n'

In [5]:
transDataDict={}
for i in range(55):
    temp=(data[col[i]]-np.mean(data[col[i]]))/(np.max(data[col[i]])-np.min(data[col[i]]))
    transDataDict[col[i]]=temp
transData=pd.DataFrame.from_dict(transDataDict)
del transDataDict

In [6]:
mergeData=transData.apply(lambda x:[x['CHI611'],x['CHI612'],x['CHI621'],x['CHI622'],
x['CHI711'],x['CHI712'],x['CHI721'],x['CHI722'],
x['CHI811'],x['CHI812'],x['CHI821'],x['CHI822'],
x['CHI911'],x['CHI912'],x['CHI921']],axis=1)
mergeData=pd.DataFrame(mergeData,columns=['CHI'])
mergeData['CHINoNan']=mergeData['CHI'].apply(lambda x:[i for i in x if not np.isnan(i)])
mergeData['CHILen']=mergeData['CHINoNan'].apply(lambda x:len(x))

In [7]:
def merge(x,col):
    if col in ['MATH','ENG']:
        x[col]=transData.apply(lambda x:[x[col+'611'],x[col+'612'],x[col+'621'],x[col+'622'],
        x[col+'711'],x[col+'712'],x[col+'721'],x[col+'722'],
        x[col+'811'],x[col+'812'],x[col+'821'],x[col+'822'],
        x[col+'911'],x[col+'912'],x[col+'921']],axis=1)
    elif col == 'PHY':
        x[col]=transData.apply(lambda x:[x[col+'811'],x[col+'812'],x[col+'821'],x[col+'822'],
        x[col+'911'],x[col+'912'],x[col+'921']],axis=1)
    elif col == 'CHE':
        x[col]=transData.apply(lambda x:[x[col+'911'],x[col+'912'],x[col+'921']],axis=1)
def NoNan(x,col):
    x[col+'NoNan']=x[col].apply(lambda x:[i for i in x if not np.isnan(i)])
    x[col+'Len']=x[col+'NoNan'].apply(lambda x:len(x))
def delOrigin(x,col):
    x.drop([col],axis=1,inplace=True)

delOrigin(mergeData,'CHI')

for i in ['MATH','ENG']:
    merge(mergeData,i)
    NoNan(mergeData,i)
    delOrigin(mergeData,i)

In [8]:
def normTest(x,col):
    if x[col+'Len']>=3:
        return stats.shapiro(x[col+'NoNan'])
    else:
        return (-1,-1)

for i in ['CHI','MATH','ENG']:
    mergeData[i+'Norm']=mergeData.apply(lambda x:normTest(x,i),axis=1)
equalStdData=mergeData[(mergeData['CHINorm'].apply(lambda x:x[1]>0.05)) & (mergeData['MATHNorm'].apply(lambda x:x[1]>0.05)) & (mergeData['ENGNorm'].apply(lambda x:x[1]>0.05))]

In [9]:
def stdTest(df):
    temp=df[['CHINoNan','MATHNoNan','ENGNoNan']].values
    stat,p=stats.levene(temp[0],temp[1],temp[2])
    return p>0.05
equalStdData['stdTest']=equalStdData.apply(stdTest,axis=1)

In [10]:
equalStdData=equalStdData[equalStdData['stdTest']]
def oneWay(df):
    temp=df[['CHINoNan','MATHNoNan','ENGNoNan']].values
    stat,p=stats.f_oneway(temp[0],temp[1],temp[2])
    return p<0.05
equalStdData['isBias']=equalStdData.apply(oneWay,axis=1)
analIndex=equalStdData.index.values

In [11]:
noEqualStdData=mergeData.loc[np.setdiff1d(np.arange(243),analIndex)].copy()
def kruskal(df):
    temp=df[['CHINoNan','MATHNoNan','ENGNoNan']].values
    stat,p=stats.kruskal(temp[0],temp[1],temp[2])
    return p<0.05
noEqualStdData['isBias']=noEqualStdData.apply(kruskal,axis=1)

In [12]:
equalStdData=pd.concat([equalStdData,noEqualStdData])

In [13]:
sexBiasCount=equalStdData.join(data['Sex'])
sexBiasCount=sexBiasCount.groupby(['Sex','isBias'])['CHINoNan'].count().reset_index()
sexBiasCount.rename(columns={'CHINoNan':'count'},inplace=True)
sexBiasCount=sexBiasCount.pivot(index='Sex',columns='isBias',values='count')
print(sexBiasCount)
stats.chi2_contingency(sexBiasCount)

isBias  False  True 
Sex                 
F          26     83
M          34     85


(0.4324735398746665,
 0.5107777093228676,
 1,
 array([[28.68421053, 80.31578947],
        [31.31578947, 87.68421053]]))

In [14]:
for i in ['CHI','MATH','ENG']:
    mergeData[i+'Mean']=mergeData[i+'NoNan'].apply(np.mean)

In [15]:
def findBias(df):
    meanCol=['CHIMean','MATHMean','ENGMean']
    dis=np.zeros(3)
    maxScoreLesson=np.argmax(df[meanCol])
    for i in range(3):
        temp=0
        for j in meanCol:
            temp+=np.abs(df[meanCol[i]]-df[j])
        dis[i]=temp
    argMax=np.argmax(dis)
    flag=(maxScoreLesson==argMax)
    return (np.max(dis),argMax,flag)
mergeData['bias']=mergeData.apply(findBias,axis=1)

In [16]:
mergeData['biasDis']=mergeData['bias'].apply(lambda x:x[0])
mergeData['biasClass']=mergeData['bias'].apply(lambda x:x[1])
mergeData['biasPostive']=mergeData['bias'].apply(lambda x:x[2])

In [17]:
biasData=equalStdData.join(mergeData[['biasDis','biasClass','biasPostive']])[['isBias','biasDis','biasClass','biasPostive']]
data['class']=data['Index'].apply(lambda x:x[0])
biasData=biasData.join(data[['Sex','class']])

In [18]:
#已经偏科的人，偏科的科目和偏科是优势还是劣势不独立
classPostiveData=biasData[biasData['isBias']>0].groupby(['biasClass','biasPostive'])['biasDis'].count().reset_index()
temp=classPostiveData.pivot(index='biasPostive',columns='biasClass',values='biasDis')
print(temp)
stats.chi2_contingency(temp)

biasClass     0   1   2
biasPostive            
False        54  24  21
True         21  25  31


(13.931159452792105,
 0.0009438156221673181,
 2,
 array([[42.1875, 27.5625, 29.25  ],
        [32.8125, 21.4375, 22.75  ]]))

In [19]:
#偏科与性别无关
classPostiveData=biasData.groupby(['isBias','Sex'])['biasDis'].count().reset_index()
temp=classPostiveData.pivot(index='isBias',columns='Sex',values='biasDis')
print(temp)
print(stats.chi2_contingency(temp))

Sex      F   M
isBias        
False   26  34
True    83  85
(0.4324735398746665, 0.5107777093228676, 1, array([[28.68421053, 31.31578947],
       [80.31578947, 87.68421053]]))


In [20]:
classPostiveData=biasData.groupby(['isBias','class'])['biasDis'].count().reset_index()
temp=classPostiveData.pivot(index='isBias',columns='class',values='biasDis')
print(temp)
print(stats.chi2_contingency(temp))

class    A   B   C   D   E   F   G
isBias                            
False   19  14   3   7  12   6   5
True    22  28  33  24  20  24  25
(19.337429517440334, 0.0036301423671016584, 6, array([[11.18181818, 11.45454545,  9.81818182,  8.45454545,  8.72727273,
         8.18181818,  8.18181818],
       [29.81818182, 30.54545455, 26.18181818, 22.54545455, 23.27272727,
        21.81818182, 21.81818182]]))


In [21]:
# 已经偏科的人，偏科科目和性别无关
#但劣势科目，性别和科目有关
classPostiveData=biasData[biasData['isBias']>0].groupby(['biasClass','Sex'])['biasDis'].count().reset_index()
temp=classPostiveData.pivot(index='Sex',columns='biasClass',values='biasDis')
print(temp)
print(stats.chi2_contingency(temp))
classPostiveData=biasData[(biasData['isBias']>0)&(biasData['biasPostive']>0)].groupby(['biasClass','Sex'])['biasDis'].count().reset_index()
temp=classPostiveData.pivot(index='Sex',columns='biasClass',values='biasDis')
print(temp)
print(stats.chi2_contingency(temp))
classPostiveData=biasData[(biasData['isBias']>0)&(biasData['biasPostive']==0)].groupby(['biasClass','Sex'])['biasDis'].count().reset_index()
temp=classPostiveData.pivot(index='Sex',columns='biasClass',values='biasDis')
print(temp)
print(stats.chi2_contingency(temp))

biasClass   0   1   2
Sex                  
F          29  22  32
M          43  22  20
(5.468418470261134, 0.06494534329967876, 2, array([[35.57142857, 21.73809524, 25.69047619],
       [36.42857143, 22.26190476, 26.30952381]]))
biasClass   0   1   2
Sex                  
F          12   8  19
M           8  16  12
(4.935208161014613, 0.08478776048478087, 2, array([[10.4 , 12.48, 16.12],
       [ 9.6 , 11.52, 14.88]]))
biasClass   0   1   2
Sex                  
F          17  14  13
M          35   6   8
(10.382438734734652, 0.005565216632344224, 2, array([[24.60215054,  9.46236559,  9.93548387],
       [27.39784946, 10.53763441, 11.06451613]]))


In [24]:
biasData.rename(columns={'biasDis':'dis','biasClass':'lesson','biasPostive':'prep'},inplace=True)
biasData.corr(method='spearman')

,isBias,dis,lesson,prep
isBias,1.000000,0.633400,0.092605,-0.076102
dis,0.633400,1.000000,0.028772,-0.058227
lesson,0.092605,0.028772,1.000000,0.259407
prep,-0.076102,-0.058227,0.259407,1.000000
